## Batch-Channel Normalization & Weight Standardization

Ref: <a href='https://arxiv.org/abs/1903.10520'>Micro-Batch Training with Batch-Channel Normalization and Weight Standardization</a>

위 논문에서는 GPU 메모리 제약으로 인해 GPU 당 1~2 장의 이미지만을 할당하여 학습을 진행하는 마이크로 배치 학습(Micro-Batch Training)에서의 Batch Normalization(BN)의 문제점을 해결하기 위해 Weight Standardization(WS)과 Batch-Channel Normalization(BCN)이라는 두 가지 기술을 제안했다. 

WS는 Lipschitz 상수를 줄임으로써 손실 경사 평탄화를 수행하기 위해 컨볼루션 층의 가중치를 standardize하며 BCN은 배치 정규화와 채널 정규화를 결합하고 컨볼루션 층의 activation에 대한 추정 통계를 활용하여 네트워크를 제거 특이점(elimination singularities)에서 멀어지도록 만든다. 

여기서 제거 특이점이란, 뉴런이 지속적으로 비활성화되는 지점을 의미하며 이는 모델의 훈련 속도를 늦추고 성능을 떨어뜨린다. BN은 배치 통계를 사용하여 활성화를 정규화함으로써 뉴런 활성화 간의 균형을 더 잘 유지하고 제거 특이점을 특징으로 하는 비활성화를 피하기 때문에 이러한 문제를 완화할 수 있다. 그러나 앞서 언급하였듯이, 배치 정규화는 작은 배치 사이즈를 사용할 때는 적은 샘플 크기로 인해 배치 통계량을 제대로 구할 수가 없다.

Layer Normalizarion(LN) 및 Group Normalization(GN)과 같은 정규화 방식은 이러한 BN의 문제점을 해결하기 위해 배치 통계가 아닌 개별 채널 내 통계 정보를 사용하는 정규화 방법이지만 이로 인해 특정 채널이 과도하게 활성화 되거나 거의 비활성화 되는 문제가 발생하게 되고 제거 특이점 문제를 야기할 수 있다.

이러한 한계를 해결하기 위해 저자들은 배치 통계를 채널 정규화에 통합한 BCN과 채널 간의 통계적 유사성을 유지하도록 가중치를 제한하여 뉴런이 지속적으로 비활성화되는 것을 방지하는 WS를 제안했다.

### Batch-Channel Normalization

BCN은 먼저 배치 평균과 분산을 추정하여 배치 정규화를 수행한 후, 특징 채널을 그룹으로 나누고 각 그룹 내에서 평균과 분산을 계산하여 채널 정규화를 수행한다.

Running mean $\hat{mu}_C$와 Running variance $\hat{\sigma}^2_C$가 momentum $r$에 대해 다음과 같을 때,

$$
\hat{\mu}_C \leftarrow (1-r)\hat{\mu}_C +  r{1\over BHW}\sum_{b,h,w}X_{b,c,h,w}
$$

$$
\hat{\sigma}^2_C \leftarrow (1-r)\hat{\sigma}^2_C +  r{1\over BHW}\sum_{b,h,w}(X_{b,c,h,w}-\hat{\mu}_C)^2
$$

배치 입력 $X\in \mathbb{R}^{B\times C\times H\times W}$에 대하여서 Estimated Batch Norm은 다음과 같다.

$$
\dot{X}_{\cdot, C,\cdot,\cdot} = \gamma_C {X_{\cdot,C,\cdot,\cdot} - \hat{\mu}_C\over \hat{\sigma}_C} + \beta_C
$$

In [3]:
import torch
from torch import nn

In [4]:
class EstimatedBatchNorm(nn.Module):
    def __init__(self, channels: int,
                 eps: float = 1e-5, momentum: float = 0.1, affine: bool = True):
        super().__init__()
        
        self.channels = channels
        self.eps = eps
        self.momentum = momentum
        self.affine = affine
        
        if self.affine:
            self.gamma = nn.Parameter(torch.ones(self.channels))
            self.beta = nn.Parameter(torch.zeros(self.channels))
        
        # Tensors for \hat{\mu}_C and \hat{\sigma}^2_C  
        self.register_buffer('exp_mean', torch.zeros(channels))
        self.register_buffer('exp_var', torch.ones(channels))
        
    def forward(self, x: torch.tensor):
        x_shape = x.shape
        
        batch_size = x_shape[0]
        assert self.channels == x_shape[1]
        
        # reshape into [batch_size, channels, n]
        x = x.view(batch_size, self.channels, -1)
        
        # update exp_mean and exp_var in training mode only
        if self.training:
            with torch.no_grad():
                # calculate the mean across first and last dimensions
                mean = x.mean(dim=[0,2])
                mean_x2 = (x ** 2).mean(dim=[0,2])
                
                var = mean_x2 - (mean ** 2)
                
                # update exponential moving avverages
                self.exp_mean = (1 - self.momentum) * self.exp_mean + self.momentum * mean
                self.exp_var = (1 - self.momentum) * self.exp_var + self.momentum * var
                
        x_norm = (x - self.exp_mean.view(1,-1,1)) / torch.sqrt(self.exp_var + self.eps).view(1,-1,1)
        
        if self.affine:
            x_norm = self.gamma.view(1,-1,1) * x_norm + self.beta.view(1,-1,1)
            
        return x_norm.view(x_shape)